## 한식

In [6]:
import time
import re

#  load library (url 이미지 다운로드)
import urllib.request
import os

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

driver = webdriver.Chrome('/usr/sbin/chromedriver')

driver.implicitly_wait(3)
driver.get('https://www.foodfly.co.kr/restaurants')

# 한식[선택]
driver.find_element_by_xpath("/html/body/header/div[3]/form/div/div[2]").click()

# 웹 스크롤 다운 구현
elem = driver.find_element_by_tag_name("body")
no_of_pagedowns = 1
while no_of_pagedowns < 40:
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.8)
    no_of_pagedowns += 1

# 팝업창 나올 시 해결
try:
    alert = driver.switch_to_alert()
    alert.accept()
    pass
except:
    pass

html = driver.page_source

soup = BeautifulSoup(html, 'lxml')

# 한식 데이터(이미지) 수집
soup = BeautifulSoup(html, 'lxml')
img = soup.select('#restaurants > .restaurant-list .restaurant_box .restaurant_box_bg')
count = 1
for i in img:
    img_url = i.get_text("url", strip=True)
    img_url_parse = img_url[4:-1]
    urllib.request.urlretrieve(img_url_parse, f'{count}.korea_food_img.jpg')
    count += 1

# 한식 데이터(식당명) 수집
korea_store_list = []
korea_store_name = soup.select('#restaurants > .restaurant-list .restaurant_box .restaurant_info .restaurant_name')
for name in korea_store_name:
    store_name = f'{name.string}'
    korea_store_list.append(store_name)

for i, name in enumerate(korea_store_list):
    print(i, name)


    
html = driver.page_source # 현재 페이지 주소를 반환    
ele_id = driver.find_element_by_id("restaurants")
html_content = ele_id.find_element_by_css_selector('div.restaurant-list').get_attribute('innerHTML')

soup = BeautifulSoup(html_content, 'lxml')

store_href = []
for url in soup:
    url_page = url.select('a')
    for url_href in url_page:
        a_href = url_href['href']
        store_href.append(a_href)


# 각 가게 링크에 접속 후 파싱
# 가게주소
store_addresss=[]
# 가게 소개
store_info_list = []
# 가게 원산지 정보
store_origin_infos = []
# 가게 최소금액
store_moneyss = []
# 가게 음식 메뉴
store_menus = []
# 음식 가격
store_prices = []
# 음식 정보
store_descriptions = []

num = 1
for p in store_href:
    # 팝업나올시 해결
    try:
        alert = driver.switch_to_alert()
        alert.accept()
        pass
    except:
        pass
    
    try:
        driver.get(p)
        print(f'---{num}번 식당파싱 시작 ---')
        num += 1
        html2 = driver.page_source
        soup = BeautifulSoup(html2, 'lxml')
    except:
        print(f'{num}번 오류')
        pass
    # 주소뽑기
    store_address = soup.select('#restaurant-show > .clearfix > .left-section .pull-left p .info-header + span')[1]
    store_address_str = store_address.contents[0]
    store_address_str_parse = store_address_str.replace(': ', '')
    store_addresss.append(store_address_str_parse)
    print(store_address_str_parse)    
    print('주소 뽑기 성공')
    print('=========================================')
    
    
    # 가게소개 뽑기
    store_info = soup.select('#restaurant-show > .restaurant-info > p')    
    description = ''
    for x in store_info:
        store_info_str = x.get_text(strip=True)
        content = store_info_str + '\n'
        description += content
    store_info_list.append(description)
    print(description)
    print('가게 소개 뽑기 성공')
    print('=========================================')
    
#     # 가게 원산지 정보 뽑기
    store_origin = soup.select_one('div.restaurant-info').find_next_sibling('div')
    origin_content = store_origin.select('p')
    
    description2 = ''
    for w in origin_content:
        store_origin_info = w.get_text(strip=True)
        content2 = store_origin_info + '\n'
        description2 += content2
    store_origin_infos.append(description2)
    print(description2)
    print('가게 원산지 뽑기 성공')
    print('=========================================')    
    
    # 가게최소금액 뽑기
    store_least_money_list = soup.select('.main-info-sub > p > span.info-header + span')
    for a in store_least_money_list:
        store_least_money = store_least_money_list[3]
        store_least_money_result = store_least_money.get_text(":", strip=True)[2:8]
        store_split = re.split(',', store_least_money_result)
        store_money = ''
        for i in store_split:
            store_money += i
    store_moneys=int(store_money)
    store_moneyss.append(store_moneys)
    print(store_moneys)
    print('최소 금액 성공')
    print('=========================================')

    store_details_menu = soup.select('.left-section > .menu-category > .row-menu')
    
    
    for menu in store_details_menu:
        # 음식 메뉴
        store_menu = menu.select_one(".pull-left").find_next('strong').get_text(strip=True)
        store_menus.append(store_menu)
        print(store_menu)
        print('음식 메뉴 성공')
        print('=========================================')

        # 음식 정보
        store_description = menu.select_one('.pull-left > span').get_text("", strip=True)
        store_descriptions.append(store_description)
        print(store_description)
        print('음식 정보 성공')
        print('=========================================')

        # 음식 가격
        store_price = menu.select_one("span.calculate-price").get_text(strip=True)
        a = re.split(',', store_price)
        aprice = ''
        for i in a:
            aprice += i
        price = int(aprice)
        store_prices.append(price)
        print(price)
        print('음식 가격 성공')
        print('=========================================')


    try:
        alert = driver.switch_to_alert()
        alert.accept()
        pass
    except:
        pass


    print('===========끝===========')



    driver.back()
    driver.implicitly_wait(12)


print('**************끝났따!!!!!!!!!!!!!!!!!*********************')
driver.close()
      
    
#     print('================== 가게 하나 끝 ==================')
    # 팝업나올시 해결

# 한식 데이터(식당명) 수집
print(korea_store_list)
# 가게주소
print(store_addresss)
# 가게 소개
print(store_info_list)
# 가게 원산지 정보
print(store_origin_infos)
# 가게 최소금액
print(store_moneyss)
# 가게 음식 메뉴
print(store_menus)
# 음식 가격
print(store_prices)
# 음식 정보
print(store_descriptions)

/home/hanyonghee/.local/share/virtualenvs/foodfly-av6wuyWs/lib/python3.6/site-packages/ipykernel_launcher.py:30: DeprecationWarning: use driver.switch_to.alert instead


0 셰플리 강남키친
1 짚신매운갈비찜 역삼점
2 스트릿 SPC스퀘어점
3 막퍼주는식당
4 직화락 역삼점
5 백억하누 강남본점
6 봉추찜닭 역삼점
7 계절밥상 강남역점
8 고요남 강남점
9 유쾌한접시 신논현점
10 춘이네닭도리탕 논현점
11 미소꽃돈
12 고요남 논현본점
13 계절밥상 강남롯데점
14 교대평상집 본점
15 빌리키친
16 나주가
17 통돼지찌개집 역삼점
18 무교동유정낙지 역삼점
19 만족오향족발 강남직영점
20 가장맛있는족발족과의동침 강남태극당점
21 장독대김치찌개 국기원점
22 한촌설렁탕 강남점
23 한옥집김치찜 역삼점
24 생태명가 본점
25 한촌설렁탕 역삼점
26 황금복국 강남점
27 채서원병천순대
28 신촌설렁탕함흥냉면 역삼점
29 쌍다리돼지불백 역삼직영점
30 죽이야기 역삼2호점
31 나주곰탕 역삼역점
32 배나무골
33 마실 역삼점
34 놀부부대찌개 역삼2호점
35 놀부부대찌개 강남씨티점
36 돈수백 역삼점
37 아낙 역삼점
38 금강정육식당
39 한촌설렁탕 역삼GS점
40 백운봉막국수 역삼점
41 봉족발
42 봉추찜닭 선릉점
43 소공동뚝배기집 교대점
44 타이반쩜
45 봉추찜닭 교대역점
46 가장맛있는족발 선릉점
47 곱창추
48 조선옥 역삼점
49 8군부대찌개&통돼지찌개집
50 생태명가
51 복돈이가마솥순대국
52 명인만두 도곡1호점
53 지니전복죽
54 신촌설렁탕 강남점
55 남서울아바이순대
56 서울전통육개장 본점
57 아빠곰탕 강남점
58 상황버섯약수삼계탕
59 송가네
60 홍대개미 선릉점
61 논현쭈꾸미
62 신선설농탕 도곡점
63 북창동순두부 서초반포점
64 곱창의전설 논현점
65 두배족발
66 바비박스 논현점
67 오성돼지
68 모자덮밥
69 유림 강남직영점
70 대독장 교대점
71 신선설농탕 서초점
72 생선밥상 어랑
73 영칼로리 논현점
74 순남시래기 교대역점
75 한신매운갈비찜 논현점
76 대독장 선릉점
77 싸움의고수 한티역점
78 영동죽집
79 대왕부대찌개
80 만족오향족발 교대점
81 장독대김치찌개 선릉점


/home/hanyonghee/.local/share/virtualenvs/foodfly-av6wuyWs/lib/python3.6/site-packages/ipykernel_launcher.py:96: DeprecationWarning: use driver.switch_to.alert instead


---1번 식당파싱 시작 ---
서울 강남구 역삼동 669-6 전원빌딩 지하 1층 
주소 뽑기 성공

[CHEFLY] 당신만을 위한 셰프의 한끼

대한민국 대표 셰프들의 요리를
온라인주문+즉시조리+즉시배송으로
원하는 시간과 장소에서 편하게 즐기실 수 있는
신개념 온라인 음식배달 서비스입니다


가게 소개 뽑기 성공

[메인메뉴_완조리]

단짠단짠 밥도둑 연어새우장 덮밥:새우 (에콰도르), 연어 (노르웨이산), 계란(국내산)
피타브래드에 싸먹는 허니갈릭 닭가슴살 시저샐러드:로메인(국내산), 닭가슴살(국내산), 계란(국내산), 베이컨(수입산), 디종머스터드(미국산)
특제간장으로 맛을 살린여수 새꼬막비빔밥 :새꼬막(국내산), 채소류(국내산, 중국산), 고춧가루(국내산)
스파이시 마살라 쉬림프를 올린 동남아풍 레드커리 :새우(베트남산), 채소류(국내산, 중국산), 레드커리페이스트(태국산), 휘핑크림(국내산)
허브향 닭다리살 구이와 매콤한 고르곤졸라 크림 곡물 리조또 : 닭다리살(브라질산), 치킨파우더(대만산), 잡곡(국내산), 고르곤졸라(이탈리아산)
매콤한 낙지 콩나물비빔밥 : 쌀(국내산), 낙지(중국산), 채소류(국내산)
그라나파다노를 곁들인 닭다리살 김치볶음밥 :김치(국내산), 쌀(국내산), 계란(국내산), 채소류(국내산), 정육(브라질산), 그라나파다노(이탈리아산), 트러플오일(이탈리아산), 굴소스(홍콩산), 두반장(중국산)
소불고기 궁중 비빔밥 '골동반' :소고기(호주산), 계란(국내산), 시금치(국내산), 튀각(국내산), 쌀(국내산), 미역(국내산), 고사리(중국산), 당근(중국산), 청포묵(중국산), 숙주(중국산), 은행(중국산)
국내산 생등심 돈가츠 덮밥 '가츠동' :돼지등심(국내산), 계란(국내산), 쌀(국내산), 채소류(국내산), 간장(일본산), 단무지(중국산), 락교(중국산)


가게 원산지 뽑기 성공
8000
최소 금액 성공
고소하고 상큼한 흑임자 알감자 샐러드
음식 메뉴 성공
버터에 노릇노릇하게 구운 알감자에 볶음 흑임자 드레싱의 고소함과 레몬 

IntegrityError: FOREIGN KEY constraint failed

In [18]:
num = 1
for col in korea_store_list:
    store= Store.objects.create(
        pk=num,
        store_category=0,
        name=col,
        store_info=store_info_list[korea_store_list.index(col)],
        origin_info=store_origin_infos[korea_store_list.index(col)],
        owner=User.objects.get(pk=2),
        least_cost=store_moneyss[korea_store_list.index(col)],
    )
    num += 1
    print('===========끝===========')
        
    Food.objects.create(
    name= store_menu,
    price=price,
    store_id=store,
    food_info=store_description,
    )

IntegrityError: UNIQUE constraint failed: store_store.id

In [39]:
len(korea_store_list)

179

In [40]:
len(store_addresss)

1

In [41]:
len(store_info_list)

1

In [5]:
len(store_origin_infos)

178

In [6]:
len(store_moneyss)

178

In [7]:
len(store_menus)

3583

In [8]:
len(store_prices)

3583

In [9]:
len(store_descriptions)

3583

In [9]:
Store.objects.all().delete()
# num = 1
# for col in korea_store_list:
#     Store.objects.create(
#         pk=num,
#         store_category=0,
#         name=col,
#         store_info=store_info_list[korea_store_list.index(col)],
#         origin_info=store_origin_infos[korea_store_list.index(col)],
#         owner=User.objects.get(pk=2),
#         least_cost=store_moneyss[korea_store_list.index(col)],
#     )
#     num += 1

#     print(col)

(1, {'store.Store': 1})

In [ ]:
# num = 1
for col in korea_store_list:
    Food.objects.create(
        name=col,
        price=store_prices[store_menus.index(col)],
        store=
    )
    num += 1

    print(col)

In [20]:
korea_store_list

['셰플리 강남키친',
 '짚신매운갈비찜 역삼점',
 '스트릿 SPC스퀘어점',
 '직화락 역삼점',
 '백억하누 강남본점',
 '봉추찜닭 역삼점',
 '계절밥상 강남역점',
 '돼지런',
 '호랑이불백',
 '다정식',
 '고요남 강남점',
 '유쾌한접시 신논현점',
 '춘이네닭도리탕 논현점',
 '미소꽃돈',
 '고요남 논현본점',
 '계절밥상 강남롯데점',
 '빌리키친',
 '여의도황소곱창',
 '나주가',
 '통돼지찌개집 역삼점',
 '무교동유정낙지 역삼점',
 '가장맛있는족발족과의동침 강남태극당점',
 '장독대김치찌개 국기원점',
 '한촌설렁탕 강남점',
 '한옥집김치찜 역삼점',
 '생태명가 본점',
 '에덴식당 강남점',
 '한촌설렁탕 역삼점',
 '황금복국 강남점',
 '채서원병천순대',
 '동남집 역삼점',
 '신촌설렁탕함흥냉면 역삼점',
 '쌍다리돼지불백 역삼직영점',
 '죽이야기 역삼2호점',
 '나주곰탕 역삼역점',
 '배나무골',
 '마실 역삼점',
 '놀부부대찌개 역삼2호점',
 '놀부부대찌개 강남씨티점',
 '돈수백 역삼점',
 '금강정육식당',
 '한촌설렁탕 역삼GS점',
 '백운봉막국수 역삼점',
 '봉족발',
 '봉추찜닭 선릉점',
 '소공동뚝배기집 교대점',
 '타이반쩜',
 '봉추찜닭 교대역점',
 '가장맛있는족발 선릉점',
 '조선옥 역삼점',
 '8군부대찌개&통돼지찌개집',
 '생태명가',
 '복돈이가마솥순대국',
 '명인만두 도곡1호점',
 '지니전복죽',
 '신촌설렁탕 강남점',
 '남서울아바이순대',
 '서울전통육개장 본점',
 '아빠곰탕 강남점',
 '상황버섯약수삼계탕',
 '오대산산채전문점',
 '송가네',
 '장수가 선능직영점',
 '홍대개미 선릉점',
 '논현쭈꾸미',
 '신선설농탕 도곡점',
 '북창동순두부 서초반포점',
 '바비박스 논현점',
 '오성돼지',
 '모자덮밥',
 '유림 강남직영점',
 '대독장 교대점',
 '신선설농탕 서초점',
 '생선밥상 어랑',
 '영칼로리 논현점',
 '순남시래기 교대

In [23]:
num = 1
for index, col in enumerate(korea_store_list):
    store=Store.objects.create(
        pk=num,
        store_category=0,
        name=col,
        store_info=store_info_list[index],
        origin_info=store_origin_infos[index],
#         origin_info=store_origin_infos[korea_store_list.index(col)],
        owner=User.objects.get(pk=2),
        least_cost=store_moneyss[index],
    )
    num += 1
    
    store_details_menu = soup.select('.left-section > .menu-category > .row-menu')
    for menu in store_details_menu:
        # 음식 메뉴
        store_menu = menu.select_one(".pull-left").find_next('strong').get_text(strip=True)
#         store_menus.append(store_menu)
        print(store_menu)
        print('음식 메뉴 성공')
        print('=========================================')
        
        # 음식 정보
        store_description = menu.select_one('.pull-left > span').get_text("", strip=True)
#         store_descriptions.append(store_description)
        print(store_description)
        print('음식 정보 성공')
        print('=========================================')
        
        # 음식 가격
        store_price = menu.select_one("span.calculate-price").get_text(strip=True)
        a = re.split(',', store_price)
        aprice = ''
        for i in a:
            aprice += i
        price = int(aprice)
#         store_prices.append(price)
        print(price)
        print('음식 가격 성공')
        print('=========================================')
        
        Food.objects.create(
            name= store_menu,
            price=price,
            store_id=store,
            food_info=store_description,
        )
    try:
        alert = driver.switch_to_alert()
        alert.accept()
        pass
    except:
        pass
    

    print('===========끝===========')
        
    
    
    driver.back()
    driver.implicitly_wait(12)
    
    
print('**************끝났따!!!!!!!!!!!!!!!!!*********************')
driver.close()


아구찜 (소, 2~3인분, 완조리)
음식 메뉴 성공
국내산 1등급 고춧가루와 마늘, 콩나물, 미나리, 생물아구, 아구내장, 미더덕 등으로 조리를 하고있습니다.
음식 정보 성공
56000
음식 가격 성공


TypeError: int() argument must be a string, a bytes-like object or a number, not 'Store'

In [28]:
store_description

'버터에 노릇노릇하게 구운 알감자에 볶음 흑임자 드레싱의 고소함과 레몬 드레싱의 상큼함을 가미했습니다.포슬포슬한 알감자와 상큼한 토마토, 아삭한 브로콜리까지!지금까지 없던 맛과 멋, 건강까지 고려한 셰플리의 야심작입니다. 소소하지만 확실한 행복을 즐겨보세요~'

In [30]:
num = 1
for index, col in enumerate(korea_store_list):
    store=Store.objects.create(
        store_category=0,
        name=col,
        store_info=store_info_list[index],
        origin_info=store_origin_infos[index],
#         origin_info=store_origin_infos[korea_store_list.index(col)],
        owner=User.objects.get(pk=2),
        least_cost=store_moneyss[index],
    )
    num += 1

IndexError: list index out of range

In [4]:
type(store_description)

str

In [8]:
store_description

'버터에 노릇노릇하게 구운 알감자에 볶음 흑임자 드레싱의 고소함과 레몬 드레싱의 상큼함을 가미했습니다.포슬포슬한 알감자와 상큼한 토마토, 아삭한 브로콜리까지!지금까지 없던 맛과 멋, 건강까지 고려한 셰플리의 야심작입니다. 소소하지만 확실한 행복을 즐겨보세요~'